In [2]:
!pip install beautifulsoup4 requests

In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [4]:
# Define the URL of the target website
url = 'https://food.ndtv.com/recipes'

# Send an HTTP request to the target website
response = requests.get(url)

In [5]:
response.status_code

200

In [6]:
html = requests.get(url).text

In [7]:
soup  = bs(html, 'lxml')

In [8]:
link_section = soup.find_all('a', class_= "tp-ttl")

In [9]:
link_section

[<a class="tp-ttl" href="https://food.ndtv.com/recipes/by-cuisine"><div class="tp-ttl-bg tp-ttl-bg2">Cuisine<svg class="vj_icn vj_right-arrow vj_ttl-arw"><use xlink:href="#vj_right-arrow"></use></svg></div></a>,
 <a class="tp-ttl" href="https://food.ndtv.com/recipes/by-festivals"><div class="tp-ttl-bg tp-ttl-bg2">Festivals<svg class="vj_icn vj_right-arrow vj_ttl-arw"><use xlink:href="#vj_right-arrow"></use></svg></div></a>,
 <a class="tp-ttl" href="https://food.ndtv.com/recipes/by-occasion"><div class="tp-ttl-bg tp-ttl-bg2">Occasion<svg class="vj_icn vj_right-arrow vj_ttl-arw"><use xlink:href="#vj_right-arrow"></use></svg></div></a>,
 <a class="tp-ttl" href="https://food.ndtv.com/recipes/userrecipes"><div class="tp-ttl-bg tp-ttl-bg2">Member Recipes<svg class="vj_icn vj_right-arrow vj_ttl-arw"><use xlink:href="#vj_right-arrow"></use></svg></div></a>,
 <a class="tp-ttl" href="https://food.ndtv.com/recipes/breakfast-recipes"><div class="tp-ttl-bg tp-ttl-bg5">Breakfast &amp; Brunch Recipes

In [10]:
all_recipes = []
for li in link_section:
    href = li.get('href')
    if href:
        url_section= href
        response = requests.get(url_section)
        html_section = requests.get(url_section).text
        soup  = bs(html_section, 'lxml')
        recipe_section = soup.find_all('a', class_= "crd_ttl")
        for re in recipe_section:
            href2=re.get('href')
            if href2:
                url_recipes=href2
                response = requests.get(url_recipes)
                html_recipe = requests.get(url_recipes).text
                soup  = bs(html_recipe, 'lxml')
                recipe_each = soup.find_all('a', class_= "crd_img")
                for each in recipe_each:
                    href3=each.get('href')
                    if href3:
                        final_url = href3
                        response = requests.get(final_url)
                        html_final = requests.get(final_url).text
                        soup  = bs(html_final, 'lxml')
                        title = soup.find('h1', class_='sp-ttl').text.strip()
                        image_tag = soup.find('img', id='story_image_main')
                        if image_tag:
                            # Try to get the 'data-original' attribute first; if not present, fall back to 'src'
                            image_link = image_tag.get('data-original') or image_tag.get('src')
                        else:
                            image_link = None  # Handle cases where the image is not found
                        times = soup.find_all('span', class_='RcpInf_crd_tx2')
                        total_cook = times[0].text.strip() if len(times) > 0 else "N/A"
                        prep_time = times[1].text.strip() if len(times) > 1 else "N/A" 
                        cook_time=times[2].text.strip() if len(times) > 2 else "N/A"
                        recipe_servings=times[3].text.strip() if len(times) > 3 else "N/A"

                        level = soup.find_all('span', class_='RcpInf_crd_tx1')
                        difficulty=level[4].text.strip() if len(level) > 4 else "N/A"

                        ingredients = soup.find_all('li', class_="RcpIngd-tp_li")
                        ingredient_list = [ingre.text.strip() for ingre in ingredients]

                        all_recipes.append({
                                'Title': title,
                                'Total Cook Time': total_cook,
                                'Prep Time': prep_time,
                                'Cook Time' : cook_time,
                                'Recipe Servings' : recipe_servings,
                                'Difficulty' : difficulty,
                                'Ingredients': ', '.join(ingredient_list), 
                                'Image Link': image_link   # Join ingredients as a comma-separated string
                                   })


In [11]:
print(all_recipes)

[{'Title': 'Eggless Banana Pancake Recipe', 'Total Cook Time': '20 mins', 'Prep Time': '05 mins', 'Cook Time': '15 mins', 'Recipe Servings': '2', 'Difficulty': 'Easy', 'Ingredients': '1 Cup  all-purpose flour, 1 tbsp sugar, 1 tsp baking powder, A pinch of salt, 1  ripe banana, mashed, 1 cup milk, 2 tbsp melted butter or oil', 'Image Link': 'https://i.ndtvimg.com/i/2015-07/pancake-625_625x350_41435922828.jpg'}, {'Title': 'Leftover Rice Pancakes Recipe', 'Total Cook Time': '25 mins', 'Prep Time': '10 mins', 'Cook Time': '15 mins', 'Recipe Servings': '2', 'Difficulty': 'Easy', 'Ingredients': '1 Cup leftover cooked rice, 1 cup all-purpose flour, 1 tbsp granulated sugar, 1 tsp baking powder, 1/2 tsp baking soda, 1/4 tsp salt, 1 cup buttermilk/milk, 1 large egg, 2 tbsp melted butter/oil, 1 tbsp vanilla extract (optional)', 'Image Link': 'https://c.ndtvimg.com/2023-07/o4pnb5j8_leftover-rice-pancakes_625x300_25_July_23.jpg'}, {'Title': 'Desi Style Masala-Cheese Hot Dog Recipe', 'Total Cook Tim

In [12]:
import csv
def save_to_csv(data, filename='recipe_titles.csv'):
    # Ensure data is being passed correctly
    if data:
        # Define the CSV header
        header = ['Title','Total Cook Time','Prep Time','Cook Time','Recipe Servings','Difficulty', 'Ingredients','Image Link']
        
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=header)
            
            # Write the header
            writer.writeheader()
            
            # Write each title to the CSV
            for row in data:
                writer.writerow(row)

In [13]:
import csv
# Save the collected recipe titles to the CSV file
save_to_csv(all_recipes)

# Check if the list is correctly populated
if all_recipes:
    print(f"Saved {len(all_recipes)} recipe titles to recipe_titles.csv")
else:
    print("No recipe titles found.")

Saved 1493 recipe titles to recipe_titles.csv


In [14]:
!pip install pandas langdetect
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Fix randomness in langdetect
DetectorFactory.seed = 0

# Load the CSV file
df = pd.read_csv('recipe_titles.csv')

# Define a function to detect the language of a given text
def detect_language(text):
    try:
        # langdetect returns 'hi' for Hindi
        return detect(str(text))
    except LangDetectException:
        return 'unknown'

# Apply the function to each row and filter out Hindi rows
filtered_df = df[~df.apply(lambda row: row.astype(str).apply(detect_language).eq('hi').any(), axis=1)]

# Save the filtered data to a new CSV file
filtered_df.to_csv('recipes1.csv', index=False)

print("Hindi rows removed successfully!")

Hindi rows removed successfully!
